# Translacja
Chodzi o to żeby przekształcić tekst, aby dało się go wyrenderować przy pomocy mojego specjalnego fontu dla systemu stenografii.
Założeniem systemu stenografii jest to, żeby nie trzeba było odrywać pióra przy pisaniu wyrazu.

Idea systemu:
1. Pisze się wokół linii środkowej ciągnąc kreskę do linii górnej lub dolnej i prawie zawsze (oprócz końca wyrazu) wracając do linii środkowej
1. Kreski mają różne (proste) kształy (w sumie 11)
1. Spółgłoski oznaczane są kreską od linii środkowej w górę lub w dół (różne spółgłoski dla lustrzanej kreski idącej w górę i w dół)
1. Samogłoski oznaczane są kreską od linii górnej lub dolnej do środkowej (ta sama samogłoska może być napisana wracając z góry jak i z dołu po przez lustrzane odbicie)
1. Jeśli występują po sobie dwie spółgłoski lub dwie samogłoski, to między nimi następuje powrót przy pomocy linii pionowej
1. Na końcu spółgłoski może dodany być jeden z dwóch akcentów (kreska i kółko/pętelka)
1. Na końcu samogłoski może dodany być akcent (mała skośna linia i powrót) który zmienia `e` na `ę`, `a` na `ą`
1. Generalnie to można pisać też samogłoski poziomo, na linii środkowej, co jest przydatne gdy samogłoska następuje po samogłosce. Jest to obsłużone przez specjalne ligatury. Póki co tylko `ie` i `ia`.

## Transkrypcja

Pierwszą fazą jest podzielenie tekstu na głoski.

In [1]:
import re

space_pattern = re.compile(r'\s+')
def cleanup(t):
    return re.sub(space_pattern, ' ', t.lower())

cleanup("Nie! 117  dąsać się-pROszę.!")

'nie! 117 dąsać się-proszę.!'

In [2]:
split_pattern = re.compile(r'(sz|cz|rz|dz|dź|dż|ch)')
def dziel(t):
    pos = 0
    for m in re.finditer(split_pattern, t):
        s = m.span()
        if m.start() != pos:
            for c in t[pos:m.start()]:
                yield c
        yield m[1]
        pos = m.end()
    if pos < len(t):
        for c in t[pos:]:
            yield c

print(list(dziel('zaszczute rzeźnickie dżdżownice dźwięczą dzisiaj dzdz ')))

['z', 'a', 'sz', 'cz', 'u', 't', 'e', ' ', 'rz', 'e', 'ź', 'n', 'i', 'c', 'k', 'i', 'e', ' ', 'dż', 'dż', 'o', 'w', 'n', 'i', 'c', 'e', ' ', 'dź', 'w', 'i', 'ę', 'cz', 'ą', ' ', 'dz', 'i', 's', 'i', 'a', 'j', ' ', 'dz', 'dz', ' ']


In [3]:
def czy_samlgloska(s):
    return s in ['a','i','e','o','y','u','ę','ą','ó']

def czy_spolgloska(s):
    return str.isalpha(s) and not czy_samlgloska(s)

Spółgłoski dzielą się na górne i dolne.

In [4]:
gorne = {
    'm',
    's','ś','sz',
    't',
    'l',
    'p',
    'k',
    'w',
    'r',
    'c','ć','cz',
    'dz','dż','dź',
    'x'
}

dolne = {
    'n','ń'
    'z','ż','ź'
    'd',
    'ł',
    'b',
    'g',
    'f',
    'j',
    'h','ch',
    'v',
    'q'
}

Po górnej spółgłosce może występować tylko górna samogłoska (oznaczona upper-case), a po dolnej - dolna.

Jeśli po spółgłosce jest spółgłoska, to między nimi musi znaleźć się linia wracająca do środkowej linii (górna `↓` lub dolna `↑`).

Tymczasowo podobnie dla samogłosek, póki nie są obsługiwane poziome formy samogłosek.

Kolejnym poziomem uproszczenia byłoby zakodowanie tego w foncie. W tym celu można użyć mechanizmu kontekstowych podstawień.
Np. jeśli górna litera jest po górnej, to należy użyć formy alternatywnej - z łącznikiem.

In [5]:
class Klasa:
    def __init__(self, c):
        self.samogloska = czy_samlgloska(c[0])
        self.spolgloska = czy_spolgloska(c[0])
        self.inna = not self.samogloska and not self.spolgloska
        self.gorna = (self.spolgloska and c in gorne) or self.inna

def ligatura(prev, c):
    return (prev, c) in [('i','e'), ('i','a'), ('i','ę'), ('i','ą')]

def polacz(iter):
    prev = ' '
    pk = Klasa(prev)
    for c in iter:
        k = Klasa(c)
        if k.samogloska:
            if pk.samogloska and not ligatura(prev, c):
                # po samogłosce bez ligatury zawsze forma górna z łącznikiem
                yield '↓'
                yield c.upper()
                k.gorna = True
            else:
                # to czy forma górna czy dolna zależy od tego jak skończyła się poprzednia litera
                k.gorna = pk.gorna
                yield c.upper() if k.gorna else c
        elif k.spolgloska and pk.spolgloska:
            # łącznik przed spółgłoską jeśli poprzedzała ją spółgłoska
            yield '↓' if pk.gorna else '↑'
            yield c
        else:
            # bez zmian
            yield c
        prev = c
        pk = k

print(list(polacz(['p','i', 'ę', 'k'].__iter__())))

['p', 'I', 'Ę', 'k']


In [6]:
def konwertuj(s):
    return polacz(dziel(cleanup(s)))

''.join(konwertuj('i tak i dzisiejszy dzień zaczął się pięknie'))

'I tAk I dzIsIEj↑szY dzIEń zaczĄł sIĘ pIĘk↓nie'

In [7]:
import io

def line_to_char(lines):
    for line in lines:
        for c in dziel(cleanup(line)):
            yield c
        yield '\n'
        

with open('felieton.txt', 'r', encoding='utf-8') as f:
    with open('felieton.out.txt', 'w', encoding='utf-8') as out:
        for c in polacz(line_to_char(f)):
            out.write(c)

## TODO
- statystyki dla par samogłosek i ligatury dla najpopularniejszych

### Pomysły
- font z kontekstowymi wariantami, żeby nie trzeba było używać skryptu pythona do wstawiania łączników i zmiany case samogłosek